# Hate Speech Detection Results

In [ ]:
import pandas as pd
import numpy as np
import re
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support
from sklearn.linear_model import LogisticRegression
import seaborn as sns
import matplotlib.pyplot as plt

## Loading Data

In [ ]:
df_tweets = pd.read_parquet('data/tweets/en/english_tweets.parquet')
df_tweets

In [ ]:
hate_a = pd.read_parquet('data/hate_speech/hate_model_a.parquet')
hate_a['id'] = hate_a['id'].astype(str)
hate_a

In [ ]:
hate_b = pd.read_parquet('data/hate_speech/hate_model_b.parquet')
hate_b['id'] = hate_b['id'].astype(str)
hate_b

In [ ]:
hate_b.rename(columns={'not-hate': 'NON_HATE', 'hate': 'HATE', 'not-hate_softmax': 'NON_HATE_softmax', 'hate_softmax': 'HATE_softmax'}, inplace=True)
hate_b

## Labeling Statistics

In [ ]:
merged_hate = hate_a.merge(hate_b, on='id', suffixes=('_a', '_b'))
merged_hate

In [ ]:
merged_hate['hate_speech_a_label'] = merged_hate['HATE_softmax_a'] >= 0.5
merged_hate['hate_speech_b_label'] = merged_hate['HATE_softmax_b'] >= 0.5

In [ ]:
merged_hate

In [ ]:
d = merged_hate.hate_speech_a_label.value_counts().rename(index={True: 'Hate', False: 'Non-Hate'})

d = d / d.sum() * 100

d = d.to_frame('Percentage').reset_index().rename(columns={'index': 'Model Output'})

ax = sns.barplot(x='Model Output', y='Percentage', data=d, palette='Blues')
ax.set_title('Model A Output')

ax.bar_label(ax.containers[0])
for t in ax.texts: t.set_text(t.get_text() + " %")

plt.savefig('plots/res/' + ax.title.get_text() + '.svg', format='svg', bbox_inches="tight")
plt.show()

In [ ]:
d = merged_hate.hate_speech_b_label.value_counts().rename(index={True: 'Hate', False: 'Non-Hate'})

d = d / d.sum() * 100

d = d.to_frame('Percentage').reset_index().rename(columns={'index': 'Model Output'})

ax = sns.barplot(x='Model Output', y='Percentage', data=d, palette='Blues')
ax.set_title('Model B Output')

ax.bar_label(ax.containers[0])
for t in ax.texts: t.set_text(t.get_text() + " %")

plt.savefig('plots/res/' + ax.title.get_text() + '.svg', format='svg', bbox_inches="tight")
plt.show()

In [ ]:
agreement = (merged_hate.hate_speech_a_label == merged_hate.hate_speech_b_label).value_counts().rename(index={True: 'Similar', False: 'Different'})

agreement = agreement / agreement.sum() * 100

agreement = agreement.to_frame('Percentage').reset_index().rename(columns={'index': 'Model Output Similarity'})

ax = sns.barplot(x='Model Output Similarity', y='Percentage', data=agreement, palette='Blues')
ax.set_title('Model A and B Output Similarity')

ax.bar_label(ax.containers[0])
for t in ax.texts: t.set_text(t.get_text() + " %")

plt.savefig('plots/res/' + ax.title.get_text() + '.svg', format='svg', bbox_inches="tight")
plt.show()

In [ ]:
print(f'{merged_hate.hate_speech_a_label.sum()} tweets labeled as hate by model a')
print(f'{merged_hate.hate_speech_b_label.sum()} tweets labeled as hate by model b')

In [ ]:
cf_matrix = confusion_matrix(merged_hate.hate_speech_a_label, merged_hate.hate_speech_b_label)
cf_matrix

ax = sns.heatmap(cf_matrix, annot=True, cmap='Blues', fmt='g')

ax.set_title('Model A and B Confusion Matrix')
ax.set_xlabel('Hate Speech B')
ax.set_ylabel('Hate Speech A')

## Ticket labels - List must be in alphabetical order
ax.xaxis.set_ticklabels(['False','True'])
ax.yaxis.set_ticklabels(['False','True'])

## Display the visualization of the Confusion Matrix.
plt.savefig('plots/res/' + ax.title.get_text() + '.svg', format='svg', bbox_inches="tight")
plt.show()

In [ ]:
cf_matrix = confusion_matrix(merged_hate.hate_speech_a_label, merged_hate.hate_speech_b_label, normalize='pred') * 100
cf_matrix

ax = sns.heatmap(cf_matrix, annot=True, cmap='Blues', fmt='.2f')
for t in ax.texts: t.set_text(t.get_text() + " %")

ax.set_title('Model A and B Confusion Matrix (B Normalized)')
ax.set_xlabel('Hate Speech B')
ax.set_ylabel('Hate Speech A')

## Ticket labels - List must be in alphabetical order
ax.xaxis.set_ticklabels(['False','True'])
ax.yaxis.set_ticklabels(['False','True'])

## Display the visualization of the Confusion Matrix.
plt.savefig('plots/res/' + ax.title.get_text() + '.svg', format='svg', bbox_inches="tight")
plt.show()

In [ ]:
cf_matrix = confusion_matrix(merged_hate.hate_speech_a_label, merged_hate.hate_speech_b_label, normalize='true') * 100
cf_matrix

ax = sns.heatmap(cf_matrix, annot=True, cmap='Blues', fmt='.2f')
for t in ax.texts: t.set_text(t.get_text() + " %")

ax.set_title('Model A and B Confusion Matrix (A Normalized)')
ax.set_xlabel('Hate Speech B')
ax.set_ylabel('Hate Speech A')

## Ticket labels - List must be in alphabetical order
ax.xaxis.set_ticklabels(['False','True'])
ax.yaxis.set_ticklabels(['False','True'])

## Display the visualization of the Confusion Matrix.
plt.savefig('plots/res/' + ax.title.get_text() + '.svg', format='svg', bbox_inches="tight")
plt.show()

In [ ]:
cf_matrix = confusion_matrix(merged_hate.hate_speech_a_label, merged_hate.hate_speech_b_label, normalize='all') * 100
cf_matrix

ax = sns.heatmap(cf_matrix, annot=True, cmap='Blues', fmt='.2f')
for t in ax.texts: t.set_text(t.get_text() + " %")

ax.set_title('Model A and B Confusion Matrix (Normalized)')
ax.set_xlabel('Hate Speech B')
ax.set_ylabel('Hate Speech A')

## Ticket labels - List must be in alphabetical order
ax.xaxis.set_ticklabels(['False','True'])
ax.yaxis.set_ticklabels(['False','True'])

## Display the visualization of the Confusion Matrix.
plt.savefig('plots/res/' + ax.title.get_text() + '.svg', format='svg', bbox_inches="tight")
plt.show()

## Sample Creation per Label

In [ ]:
df_tweets_classified = df_tweets.merge(merged_hate, on='id')[['id', 'text', 'NON_HATE_a', 'HATE_a', 'NON_HATE_softmax_a', 'HATE_softmax_a', 'NON_HATE_b', 'HATE_b', 'NON_HATE_softmax_b', 'HATE_softmax_b', 'hate_speech_a_label', 'hate_speech_b_label']]
df_tweets_classified

## Load Sample Labels

### TT

In [ ]:
labeled_tt = pd.read_csv('data/hate_speech/hate_speech_classification_samples/TT_labelled.csv')
labeled_tt.head()

In [ ]:
labeled_tt['label'] = labeled_tt['Label'] == 1

In [ ]:
ax = sns.scatterplot(data=labeled_tt, x='HATE_softmax_a', y='HATE_softmax_b', hue='label')
ax.set_title('Tweets Labeled as Hate Speech by Model A and Model B')
plt.savefig('plots/res/' + ax.title.get_text() + '.svg', format='svg', bbox_inches="tight")
plt.show()
#plt.ylim(0, 1)
#plt.xlim(0, 1)

### TF

In [ ]:
labeled_tf = pd.read_csv('data/hate_speech/hate_speech_classification_samples/TF_labelled.csv')
labeled_tf.head()

In [ ]:
labeled_tf['label'] = labeled_tf['Label'] == 1

In [ ]:
ax = sns.scatterplot(data=labeled_tf, x='HATE_softmax_a', y='HATE_softmax_b', hue='label')
ax.set_title('Tweets Labeled as Hate Speech by Model A\nand Non-Hate Speech by Model B')
plt.savefig('plots/res/' + ax.title.get_text() + '.svg', format='svg', bbox_inches="tight")
plt.show()
#plt.ylim(0, 1)
#plt.xlim(0, 1)

### FT

In [ ]:
labeled_ft = pd.read_csv('data/hate_speech/hate_speech_classification_samples/FT_labelled.csv')
labeled_ft.head()

In [ ]:
labeled_ft['label'] = labeled_ft['Label'] == 1

In [ ]:
ax = sns.scatterplot(data=labeled_ft, x='HATE_softmax_a', y='HATE_softmax_b', hue='label')
ax.set_title('Tweets Labeled as Non-Hate Speech by Model A\nand Hate Speech by Model B')
plt.savefig('plots/res/' + ax.title.get_text() + '.svg', format='svg', bbox_inches="tight")
plt.show()
#plt.ylim(0, 1)
#plt.xlim(0, 1)

### FF

In [ ]:
labeled_ff = pd.read_csv('data/hate_speech/hate_speech_classification_samples/FF_labelled.csv')
labeled_ff.head()

In [ ]:
labeled_ff['label'] = labeled_ff['Label'] == 1

In [ ]:
ax = sns.scatterplot(data=labeled_ff, x='HATE_softmax_a', y='HATE_softmax_b', hue='label')
ax.set_title('Tweets Labeled as Non-Hate Speech by Model A and Model B')
plt.savefig('plots/res/' + ax.title.get_text() + '.svg', format='svg', bbox_inches="tight")
plt.show()
#plt.ylim(0, 1)
#plt.xlim(0, 1)

## Model Stats

In [ ]:
results = pd.concat([labeled_ff, labeled_ft, labeled_tf, labeled_tt], ignore_index=True)
results.head()

### Scatters

In [ ]:
results.label.value_counts()

In [ ]:
ax = sns.scatterplot(data=results, x='HATE_softmax_a', y='HATE_softmax_b', hue='label')
ax.set_title('Tweets Scatter Using Softmax Scores Outputs\nFrom Models A and B')
plt.savefig('plots/res/' + ax.title.get_text() + '.svg', format='svg', bbox_inches="tight")
plt.show()

In [ ]:
ax = sns.scatterplot(data=results, x='HATE_a', y='HATE_b', hue='label')
#ax.set_title('Tweets Scatter Using Softmax Scores Outputs\nFrom Models A and B')
#plt.savefig('plots/res/' + ax.title.get_text() + '.svg', format='svg', bbox_inches="tight")
plt.show()

### Stats

In [ ]:
res_stats = pd.DataFrame()

#### Model A

In [ ]:
model_a_results = results[['id', 'NON_HATE_softmax_a', 'hate_speech_a_label', 'label']]
model_a_results

In [ ]:
precision, recall, f1_score, support = precision_recall_fscore_support(model_a_results.label, model_a_results.hate_speech_a_label, average='weighted')

res = {'model': 'A',
       'precision': precision,
       'recall': recall,
       'f1_score': f1_score}

res_stats = pd.concat([res_stats, pd.DataFrame([res])], ignore_index=True).drop_duplicates(subset=['model'], keep='last')
res_stats

In [ ]:
cf_matrix = confusion_matrix(model_a_results.label, model_a_results.hate_speech_a_label)
cf_matrix

ax = sns.heatmap(cf_matrix, annot=True, cmap='Blues', fmt='g')

ax.set_title('Model A Confusion Matrix')
ax.set_xlabel('Model Prediction')
ax.set_ylabel('Label')

## Ticket labels - List must be in alphabetical order
ax.xaxis.set_ticklabels(['False','True'])
ax.yaxis.set_ticklabels(['False','True'])

## Display the visualization of the Confusion Matrix.
plt.savefig('plots/res/' + ax.title.get_text() + '.svg', format='svg', bbox_inches="tight")
plt.show()

#### Model B

In [ ]:
model_b_results = results[['id', 'NON_HATE_softmax_b', 'hate_speech_b_label', 'label']]
model_b_results

In [ ]:
precision, recall, f1_score, support = precision_recall_fscore_support(model_b_results.label, model_b_results.hate_speech_b_label, average='weighted')

res = {'model': 'B',
       'precision': precision,
       'recall': recall,
       'f1_score': f1_score}

res_stats = pd.concat([res_stats, pd.DataFrame([res])], ignore_index=True).drop_duplicates(subset=['model'], keep='last')
res_stats

In [ ]:
cf_matrix = confusion_matrix(model_b_results.label, model_b_results.hate_speech_b_label)
cf_matrix

ax = sns.heatmap(cf_matrix, annot=True, cmap='Blues', fmt='g')

ax.set_title('Model B Confusion Matrix')
ax.set_xlabel('Model Prediction')
ax.set_ylabel('Label')

## Ticket labels - List must be in alphabetical order
ax.xaxis.set_ticklabels(['False','True'])
ax.yaxis.set_ticklabels(['False','True'])

## Display the visualization of the Confusion Matrix.
plt.savefig('plots/res/' + ax.title.get_text() + '.svg', format='svg', bbox_inches="tight")
plt.show()

#### Joint Model

In [ ]:
model_j_results = results[['id', 'label']].copy()
model_j_results['pred'] = results['hate_speech_a_label'] & results['hate_speech_b_label']
model_j_results

In [ ]:
precision, recall, f1_score, support = precision_recall_fscore_support(model_j_results.label, model_j_results.pred, average='weighted')

res = {'model': 'Intersection',
       'precision': precision,
       'recall': recall,
       'f1_score': f1_score}

res_stats = pd.concat([res_stats, pd.DataFrame([res])], ignore_index=True).drop_duplicates(subset=['model'], keep='last')
res_stats

In [ ]:
cf_matrix = confusion_matrix(model_j_results.label, model_j_results.pred)
cf_matrix

ax = sns.heatmap(cf_matrix, annot=True, cmap='Blues', fmt='g')

ax.set_title('Models Conjunction Confusion Matrix')
ax.set_xlabel('Model Prediction')
ax.set_ylabel('Label')

## Ticket labels - List must be in alphabetical order
ax.xaxis.set_ticklabels(['False','True'])
ax.yaxis.set_ticklabels(['False','True'])

## Display the visualization of the Confusion Matrix.
plt.savefig('plots/res/' + ax.title.get_text() + '.svg', format='svg', bbox_inches="tight")
plt.show()

#### Union Model

In [ ]:
model_u_results = results[['id', 'label']].copy()
model_u_results['pred'] = results['hate_speech_a_label'] | results['hate_speech_b_label']
model_u_results

In [ ]:
precision, recall, f1_score, support = precision_recall_fscore_support(model_u_results.label, model_u_results.pred, average='weighted')

res = {'model': 'Union',
       'precision': precision,
       'recall': recall,
       'f1_score': f1_score}

res_stats = pd.concat([res_stats, pd.DataFrame([res])], ignore_index=True).drop_duplicates(subset=['model'], keep='last')
res_stats

In [ ]:
cf_matrix = confusion_matrix(model_u_results.label, model_u_results.pred)
cf_matrix

ax = sns.heatmap(cf_matrix, annot=True, cmap='Blues', fmt='g')

ax.set_title('Models Disjunction Confusion Matrix')
ax.set_xlabel('Model Prediction')
ax.set_ylabel('Label')

## Ticket labels - List must be in alphabetical order
ax.xaxis.set_ticklabels(['False','True'])
ax.yaxis.set_ticklabels(['False','True'])

## Display the visualization of the Confusion Matrix.
plt.savefig('plots/res/' + ax.title.get_text() + '.svg', format='svg', bbox_inches="tight")
plt.show()

#### Product Model

In [ ]:
model_p_results = results[['id', 'label']].copy()
model_p_results['pred'] = np.sqrt(results['NON_HATE_softmax_a'] * results['NON_HATE_softmax_b']) >= 0.5
model_p_results

In [ ]:
precision, recall, f1_score, support = precision_recall_fscore_support(model_p_results.label, model_p_results.pred, average='weighted')

res = {'model': 'Product',
       'precision': precision,
       'recall': recall,
       'f1_score': f1_score}

res_stats = pd.concat([res_stats, pd.DataFrame([res])], ignore_index=True).drop_duplicates(subset=['model'], keep='last')
res_stats

In [ ]:
cf_matrix = confusion_matrix(model_p_results.label, model_p_results.pred)
cf_matrix

ax = sns.heatmap(cf_matrix, annot=True, cmap='Blues', fmt='g')

ax.set_title('Models Softmax Product Confusion Matrix')
ax.set_xlabel('Model Prediction')
ax.set_ylabel('Label')

## Ticket labels - List must be in alphabetical order
ax.xaxis.set_ticklabels(['False','True'])
ax.yaxis.set_ticklabels(['False','True'])

## Display the visualization of the Confusion Matrix.
plt.savefig('plots/res/' + ax.title.get_text() + '.svg', format='svg', bbox_inches="tight")
plt.show()

#### Random Forest Model

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [ ]:
X = results[['NON_HATE_a', 'HATE_a', 'NON_HATE_b', 'HATE_b']].values
y = results['label'].values

In [ ]:
_test_precision, _test_recall, _test_f1_score = [], [], []
_precision, _recall, _f1_score = [], [], []
test_preds_cmatrix = []
for i in tqdm(range(2000)):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
    classifier = RandomForestClassifier(n_estimators=100, max_depth=5).fit(X_train, y_train)
    test_pred = classifier.predict(X_test)
    
    test_precision, test_recall, test_f1_score, test_support = precision_recall_fscore_support(y_test, test_pred, average='weighted', zero_division=0)
    
    _test_precision.append(test_precision)
    _test_recall.append(test_recall)
    _test_f1_score.append(test_f1_score)
    
    cmatrix = confusion_matrix(y_test, test_pred)
    test_preds_cmatrix.append(cmatrix)

In [ ]:
sns.displot(_test_precision, kde=True, height=4, aspect=1.5, bins=20, stat='probability')
title = plt.title('Random Forest Precision Distribution on Test Sets')
plt.savefig('plots/res/' + title.get_text() + '.svg', format='svg', bbox_inches="tight")
plt.show()

In [ ]:
sns.displot(_test_recall, kde=True, height=4, aspect=1.5, bins=20, stat='probability')
title = plt.title('Random Forest Recall Distribution on Test Sets')
plt.savefig('plots/res/' + title.get_text() + '.svg', format='svg', bbox_inches="tight")
plt.show()

In [ ]:
sns.displot(_test_f1_score, kde=True, height=4, aspect=1.5, bins=20, stat='probability')
title = plt.title('Random Forest F1-Score Distribution on Test Sets')
plt.savefig('plots/res/' + title.get_text() + '.svg', format='svg', bbox_inches="tight")
plt.show()

In [ ]:
res = {'model': 'Random Forest*',
       'precision': np.mean(_test_precision),
       'recall': np.mean(_test_recall),
       'f1_score': np.mean(_test_f1_score)}

res_stats = pd.concat([res_stats, pd.DataFrame([res])], ignore_index=True).drop_duplicates(subset=['model'], keep='last')
res_stats

In [ ]:
res_stats.round(4)

In [ ]:
print(res_stats.round(3).to_latex(index=None))